# Quantify **Cell Region Morphology** - part 2.4
--------------------

## **OBJECTIVE** 
### <input type="checkbox"/> Quantify ***cell region*** morphology
In this notebook, the logic for quantifying cell regions - ***cell***, ***nucleus***, and ***cytoplasm*** - composition (how much of each region is present) and **morphology** (region size and shape) is outlined.


---------
## **Organelle Morphology**

### summary of steps

🛠️ **BUILD FUNCTION PROTOTYPE**

- **`0`** - Apply Cell Mask *(preliminary step)*

- **`1`** - Build the list of measurements we want to include from regionprops 

- **`2`** - Add additional measurements as *"extra_properties"* with custom functions

    - define a function to retrieve the standard deviation of the region's intensity values

- **`3`** - Run regionprops and export values as a pandas dataframe

- **`4`** - Add additional measurements
    - surface area
    - surface area to volume ratio

⚙️ **EXECUTE FUNCTION PROTOTYPE**

- Define `_get_org_morphology_3D` function
- Run `_get_org_morphology_3D` function
- Compare to finalized `get_org_morphology_3D` function

## **IMPORTS**

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block loads all of the necessary python packages and functions you will need for this notebook.

In [ ]:
from pathlib import Path
import os

import napari
from napari.utils.notebook_display import nbscreenshot

from skimage.measure import (regionprops, regionprops_table)

from infer_subc.core.file_io import (read_czi_image,
                                     import_inferred_organelle,
                                     list_image_files,
                                     sample_input_quant)

from infer_subc.core.img import *
from infer_subc.utils.stats import *
from infer_subc.utils.stats import (_assert_uint16_labels)
from infer_subc.utils.stats_helpers import *
from infer_subc.organelles import * 

%load_ext autoreload
%autoreload 2

## **LOAD AND READ IN IMAGE FOR PROCESSING**
> ###### 📝 **Specifically, this will include the raw image and the outputs from segmentation**

#### &#x1F6D1; &#x270D; **User Input Required:**

In [ ]:
### USER INPUT REQUIRED ###
# If using the sample data, select which cell type you would like analyze ("neuron" or "astrocyte"):
# If not using the sample data, set cell_type to None
sample_data_type = "neuron"

# Specify which file you'd like to segment from the img_file_list
test_img_n = 0

# If you are not using the sample data, please edit "USER SPECIFIED" as necessary.
## Define the path to the directory that contains the input image folder.
data_root_path = Path("USER SPECIFIED")

# Specify the file type of your raw data that will be analyzed. Ex) ".czi" or ".tiff"
raw_img_type = "USER SPECIFIED"

## Specify which subfolder that contains the input data and the input data file extension
in_data_path = data_root_path / "USER SPECIFIED"

## Specify which subfolder contains the segmentation outputs and their file type
seg_data_path = data_root_path / "USER SPECIFIED"
seg_img_type = "USER SPECIFIED"

## Specify the output folder to save the segmentation outputs if.
## If its not already created, the code below will creat it for you
out_data_path = data_root_path / "USER SPECIFIED"

#### &#x1F3C3; **Run code; no user input required**

In [ ]:
# If sample_data_type is set to "neuron" or "astrocyte", then the sample data is used and the directories are set
if sample_data_type != None:
    data_root_path, raw_img_type, seg_img_type, in_data_path, seg_data_path, out_data_path = sample_input_quant(sample_data_type)

In [ ]:
if not Path.exists(out_data_path):
    Path.mkdir(out_data_path)
    print(f"making {out_data_path}")

raw_file_list = list_image_files(in_data_path, raw_img_type)
seg_file_list = list_image_files(seg_data_path, seg_img_type)
# pd.set_option('display.max_colwidth', None)
# pd.DataFrame({"Image Name":img_file_list})

In [ ]:
raw_img_name = raw_file_list[test_img_n]

raw_img_data, raw_meta_dict = read_czi_image(raw_img_name)

channel_names = raw_meta_dict['name']
img = raw_meta_dict['metadata']['aicsimage']
scale = raw_meta_dict['scale']
channel_axis = raw_meta_dict['channel_axis']

In [ ]:
## For each import, change the string to match the suffix on the segmentation files (i.e., the stuff following the "-")

# masks
masks_seg_names = ['masks','masks_A', 'masks_B']
for m in masks_seg_names:
    if m in [i.stem.split("-")[-1] for i in seg_file_list]:
        mask_seg = import_inferred_organelle(m, raw_meta_dict, seg_data_path, seg_img_type)
        nuc_seg, cell_seg, cyto_seg = mask_seg
        break

if 'nuc' in [i.stem.split("-")[-1] for i in seg_file_list]:
    nuc_seg = import_inferred_organelle("nuc", raw_meta_dict, seg_data_path, seg_img_type)
    cell_seg = import_inferred_organelle("cell", raw_meta_dict, seg_data_path, seg_img_type)
    cyto_seg = import_inferred_organelle("cyto", raw_meta_dict, seg_data_path, seg_img_type)

#organelles
lyso_seg = import_inferred_organelle("lyso", raw_meta_dict, seg_data_path, seg_img_type)
mito_seg = import_inferred_organelle("mito", raw_meta_dict, seg_data_path, seg_img_type)
golgi_seg = import_inferred_organelle("golgi", raw_meta_dict, seg_data_path, seg_img_type)
perox_seg = import_inferred_organelle("perox", raw_meta_dict, seg_data_path, seg_img_type)
ER_seg = import_inferred_organelle("ER", raw_meta_dict, seg_data_path, seg_img_type)
LD_seg = import_inferred_organelle("LD", raw_meta_dict, seg_data_path, seg_img_type)

-------------------------
## **Visualize with `napari`**

In [ ]:
viewer = napari.Viewer()

In [ ]:
viewer.add_image(raw_img_data)
viewer.add_image(cell_seg, colormap='gray', opacity=0.3, blending ='additive')
viewer.add_image(nuc_seg, colormap='blue', blending ='additive')
viewer.add_image(cyto_seg, colormap='magenta', blending ='additive')

nbscreenshot(viewer, canvas_only=True)

In [ ]:
viewer.close()

-------------------------
# **regionprops**

To measure the amount, size, and shape of the cell regions - ***cell***, ***nucleus***, and ***cytoplasm***, we will utilize `skimage.measure.regionprops`. These measurements can be collected based on pixel/voxel units (assuming the image is isotropic in all dimensions) and or "real-world" units (e.g., microns). Since most confocal microscope images are anisotropic (mostly with respect to the Z dimension), we will preferentially utilize real-world units. Luckily, regionprops>=0.20.0 has incorporated a spacing parameter that can handle anisotropic data.

We will utilize the same concepts outlined in notebook 1.1_organelle_morphology.ipynb.

# ***BUILD FUNCTION PROTOTYPE***

## **`0` - Apply Cell Mask *(preliminary step)***
To ensure we are performing single cell analysis, we will apply the cell segmentation as a mask.

In [ ]:
nuc_masked = apply_mask(nuc_seg, cell_seg)

## **`1` - Build the list of measurements we want to include from regionprops**

In [ ]:
# start with LABEL
test_properties = ["label"]

# add position
test_properties = test_properties + ["centroid", "bbox"]

# add area
test_properties = test_properties + ["area", "equivalent_diameter"] # "num_pixels", 

# add shape measurements
test_properties = test_properties + ["extent", "euler_number", "solidity", "axis_major_length"] # "feret_diameter_max", "axis_minor_length"]

# add intensity values (used for quality checks only)
test_properties = test_properties + ["min_intensity", "max_intensity", "mean_intensity"]

## **`2` - Add additional measurements as *"extra_properties"* with custom functions**

- define a function to retrieve the standard deviation of the region's intensity values

In [ ]:
def standard_deviation_intensity(region, intensities):
    return np.std(intensities[region])

test_extra_properties = [standard_deviation_intensity]

## **`3` - Run regionprops and export values as a pandas dataframe**

In [ ]:
# regionprops wants the intensity image in XYZ instead of ZYX order
test_intensity_input = np.moveaxis(raw_img_data, 0, -1)

test_props = regionprops_table(label_image=nuc_masked, 
                               intensity_image=test_intensity_input, 
                               properties=test_properties, 
                               extra_properties=test_extra_properties,
                               spacing=scale)

test_props_table = pd.DataFrame(test_props)

In [ ]:
test_region_name = 'nuc'

test_props_table.insert(0, "object", test_region_name)
test_props_table.rename(columns={"area": "volume"}, inplace=True)

round_scale = (round(scale[0], 4), round(scale[1], 4), round(scale[2], 4))
test_props_table.insert(loc=2, column="scale", value=f"{round_scale}")

In [ ]:
#renaming intensity quantification with the channel names
test_channel_name = ["nuc", "lyso", "mito", "golgi", "perox", "ER", "LD", "residual"]

test_rename_dict = {}
for test_col in test_props_table.columns:
    for test_idx, test_name in enumerate(test_channel_name):
        if test_col.endswith(f"intensity-{test_idx}"):
            test_rename_dict[f"{test_col}"] = f"{test_col[:-1]}{test_name}_ch"

test_props_renames = test_props_table.rename(columns=test_rename_dict)

test_props_renames

## **`4` - Add additional measurements**

- surface area
- surface area to volume ratio

In [ ]:
# props["surface_area"] = surface_area_from_props(nuc_seg, props)
test_surface_area_tab = pd.DataFrame(surface_area_from_props(nuc_masked, test_props, scale))

test_props_renames.insert(12, "surface_area", test_surface_area_tab)
test_props_renames.insert(14, "SA_to_volume_ratio", test_props_renames["surface_area"].div(test_props_renames["volume"]))

pd.set_option('display.max_columns', None)

test_props_renames

# ***EXECUTE FUNCTION PROTOTYPE***

## **Define `_get_org_morphology_3D` function**

Based on the _prototyping_ above define the function to quantify amount, size, and shape of the cell regions.

In [ ]:
def _get_region_morphology_3D(region_seg: np.ndarray, 
                              region_name: str,
                              intensity_img: np.ndarray, 
                              channel_names: [str],
                              mask: np.ndarray, 
                              scale: Union[tuple, None]=None) -> Tuple[Any, Any]:
    """
    Parameters
    ------------
    segmentation_img:
        a list of all 3d np.ndarray images of the segemented cell regions (e.g., whole cell, nucleus, cytoplasm, etc.)
    names:
        names or nicknames for the cell regions being analyzed
    intensity_img:
        a 3d np.ndarray image of the "raw" florescence intensity the segmentation was based on; for our use, this is the raw image with all the channels
        we will measure the intensity within the cell region being analyzed
    mask:
        a 3d np.ndarray image of the cell mask (or other mask of choice); used to create a "single cell" analysis

    Returns
    -------------
    pandas dataframe of containing regionprops measurements (columns) for each object in the segmentation image (rows) and the regionprops object

    """
    if len(channel_names) != intensity_img.shape[0]:
        ValueError("You have not provided a name for each channel in the intensity image. Make sure there is a channel name for each channel in the intensity image.")
    
    ###################################################
    ## MASK THE REGION OBJECTS THAT WILL BE MEASURED
    ###################################################
    # in case we sent a boolean mask (e.g. cyto, nucleus, cellmask)
    input_labels = _assert_uint16_labels(region_seg)

    input_labels = apply_mask(input_labels, mask)

    ##########################################
    ## CREATE LIST OF REGIONPROPS MEASUREMENTS
    ##########################################
    # start with LABEL
    properties = ["label"]
    # add position
    properties = properties + ["centroid", "bbox"]
    # add area
    properties = properties + ["area", "equivalent_diameter"] # "num_pixels", 
    # add shape measurements
    properties = properties + ["extent", "euler_number", "solidity", "axis_major_length"] # ,"feret_diameter_max", , "axis_minor_length"]
    # add intensity values (used for quality checks)
    properties = properties + ["min_intensity", "max_intensity", "mean_intensity"]

    #######################
    ## ADD EXTRA PROPERTIES
    #######################
    def standard_deviation_intensity(region, intensities):
        return np.std(intensities[region])

    extra_properties = [standard_deviation_intensity]

    ##################
    ## RUN REGIONPROPS
    ##################
    intensity_input = np.moveaxis(intensity_img, 0, -1)

    rp = regionprops(input_labels, 
                    intensity_image=intensity_input, 
                    extra_properties=extra_properties, 
                    spacing=scale)

    props = regionprops_table(label_image=input_labels, 
                              intensity_image=intensity_input, 
                              properties=properties, 
                              extra_properties=extra_properties,
                              spacing=scale)

    props_table = pd.DataFrame(props)
    props_table.insert(0, "object", region_name)
    props_table.rename(columns={"area": "volume"}, inplace=True)

    if scale is not None:
        round_scale = (round(scale[0], 4), round(scale[1], 4), round(scale[2], 4))
        props_table.insert(loc=2, column="scale", value=f"{round_scale}")
    else: 
        props_table.insert(loc=2, column="scale", value=f"{tuple(np.ones(region_seg.ndim))}") 

    rename_dict = {}
    for col in props_table.columns:
        for idx, name in enumerate(channel_names):
            if col.endswith(f"intensity-{idx}"):
                rename_dict[f"{col}"] = f"{col[:-1]}{name}_ch"

    props_table = props_table.rename(columns=rename_dict)

    ##################################################################
    ## RUN SURFACE AREA FUNCTION SEPARATELY AND APPEND THE PROPS_TABLE
    ##################################################################
    surface_area_tab = pd.DataFrame(surface_area_from_props(input_labels, props, scale))

    props_table.insert(12, "surface_area", surface_area_tab)
    props_table.insert(14, "SA_to_volume_ratio", props_table["surface_area"].div(props_table["volume"]))

    ################################################################
    ## ADD SKELETONIZATION OPTION FOR MEASURING LENGTH AND BRANCHING
    ################################################################
    #  # ETC.  skeletonize via cellprofiler /Users/ahenrie/Projects/Imaging/CellProfiler/cellprofiler/modules/morphologicalskeleton.py
    #         if x.volumetric:
    #             y_data = skimage.morphology.skeletonize_3d(x_data)
    # /Users/ahenrie/Projects/Imaging/CellProfiler/cellprofiler/modules/measureobjectskeleton.py

    return props_table

## **Run `_get_org_morphology_3D` function**

In [ ]:
region_seg = nuc_seg
region_name = 'nuc'
intensity_img = raw_img_data
channel_names = test_channel_name
mask = cell_seg 
scale = scale

# with scale
nuc_table = _get_region_morphology_3D(region_seg=region_seg, 
                                      region_name=region_name,
                                      intensity_img=intensity_img,
                                      channel_names=channel_names,
                                      mask=mask,
                                      scale=scale)
nuc_table

In [ ]:
nuc_table.equals(test_props_renames)

## **Compare to finalized `get_org_morphology_3D` function**

In [ ]:
from infer_subc.utils.stats import get_region_morphology_3D

nuc_table_final = get_region_morphology_3D(region_seg=region_seg, 
                                            region_name=region_name,
                                            intensity_img=intensity_img,
                                            channel_names=channel_names,
                                            mask=mask,
                                            scale=scale)

In [ ]:
nuc_table.equals(nuc_table_final)